# Price Discovery Project Using Agents

This is a notebook that contains the experimentation of prioce discovery using agents.

## Import Libraries

In [1]:
import os
import re
import json
from typing import Any, Union


import PIL
from dotenv import load_dotenv


from qdrant_client import QdrantClient
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.agents import AgentExecutor, Tool, create_react_agent
from langchain.agents.agent import AgentAction
from langchain_core.callbacks.base import BaseCallbackHandler
from langchain_core.messages import HumanMessage
from langchain_community.llms import HuggingFaceEndpoint
from langchain_community.utilities import SerpAPIWrapper
from langchain_community.vectorstores import Chroma, Qdrant
from langchain_experimental.open_clip import OpenCLIPEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.tools.retriever import create_retriever_tool
from langchain_core.output_parsers import JsonOutputParser
from typing import Any, Optional, Type
from langchain.callbacks.manager import (
    CallbackManagerForToolRun,
)
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool

## Specify Variables

In [2]:
# Load environment variables
load_dotenv()

# Specify variables
model_name = "ViT-B-32"
checkpoint = "openai"
embedding_model = OpenCLIPEmbeddings(model_name=model_name, checkpoint=checkpoint)

# db_dir = "db"

## Create Vector DB to store data

**In the case of creating a new db locally**

In [3]:
# Instantiate DB
# text_db = Chroma(persist_directory=db_dir, embedding_function=embedding_model, collection_name="apparel-collection")
# images_db = Chroma(persist_directory=db_dir, embedding_function=embedding_model, collection_name="img-collection")

Then add data to the dbs. Utilize the directory loaders for this

An example is shown below:
```python

text_loader = CSVLoader(file_path="./example_data.csv")

text_data = text_loader.load()

image_loader = DirectoryLoader('folder_with_images', glob="**/*.jpg", show_progress=True)
image_data = loader.load()
```

for reference on the [csv reader](https://python.langchain.com/docs/integrations/document_loaders/csv)

for reference on the [file directory reader](https://python.langchain.com/docs/modules/data_connection/document_loaders/file_directory)

In [4]:
# Add data to db

# text_db.add_documents(text_data)
# images_db.add_images(image_data)

**If DB is already initialized**

In [26]:
# Instantiate DB: Change the persistent directory to the location of your database
# text_db = Chroma(persist_directory="../data/databases/text_db", embedding_function=embedding_model, collection_name="Text_Store")
# images_db = Chroma(persist_directory="../data/databases/images_db", embedding_function=embedding_model, collection_name="Image_Store")

**Using Qdrant DataBase on Cloud**

In [3]:
# Instantiate Client Variables
server_url = (
    "https://1883d972-3d1a-4e6d-a846-86b9952741cd.us-east4-0.gcp.cloud.qdrant.io/:6333"
)
qdrant_api_key = os.getenv("VECTORSTORE_API_KEY")

In [4]:
# Instantiate client
client = QdrantClient(server_url, api_key=qdrant_api_key)

In [5]:
# Create sample vector
query_vector = embedding_model.embed_query(
    "Women's Lightweight Open-Front Cardigan Sweater"
)

In [7]:
# Do a search to see if the vector db has been instantialized
client.search("apparel-collection", query_vector=query_vector, limit=2)

ResponseHandlingException: timed out

**If you want to create a new db remotely**

Refer to this [documentation](https://python.langchain.com/docs/integrations/vectorstores/qdrant) for more information: 

## Create agent for retrieval

**Create Retriever from local database**

In [72]:
# Instantiate retrievers from db: This is for local DBs

# text_retriever = text_db.as_retriever()
# images_retriever = images_db.as_retriever()

In [28]:
# text_retriever.get_relevant_documents("heels for women")

[Document(page_content="Men's Superlite Stripe 3 No Show Socks (3-Pair)", metadata={'price': '12.52'}),
 Document(page_content='Nike 3-pk Dri-fit Cushioned Crew Socks for MEN', metadata={'price': '39.5'}),
 Document(page_content='Nike Mens 3-Pack Jordan Jumpman No-Show Socks', metadata={'price': '33'}),
 Document(page_content="Men's Classic Trefoil Superlite No Show Socks (6-Pair)", metadata={'price': '10'})]

**Instatiate the llms**

In [5]:
# Mixtral

repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

mixtral = HuggingFaceEndpoint(repo_id=repo_id, temperature=0.3)

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\Spectra\.cache\huggingface\token
Login successful


In [32]:
# Setup google api for gemini
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

gemini = ChatGoogleGenerativeAI(
    model="gemini-pro", google_api_key=GOOGLE_API_KEY, temperature=0
)
gemini_vision = ChatGoogleGenerativeAI(
    model="gemini-pro-vision", google_api_key=GOOGLE_API_KEY, temperature=0.1
)

**Instantiate prompts**

In [7]:
# Create prompts
test_prompt = """
    
As an experienced product analyst adept at accurately determining product price ranges,utilize the available tools {tools} to answer the question asked. 
Return ONLY a reasonable, accurate and concise price range for my product after analysing its current pricing
and that of a similar products. 

    Use the following format:

    Question: the input question you must answer
    Thought: you should always think about what to do
    Action: the action to take, should be one of [{tool_names}]
    Action Input: the input to the action
    Observation: the result of the action
    ... (this Thought/Action/Action Input/Observation can repeat N times). If no result is found use your own knowledge
    Thought: I now know the final answer
    Final Answer: the final answer to the original input question

    Begin!

    Question: {input}
    Thought: {agent_scratchpad}
"""

test_prompt_one = """    
As an experienced product analyst adept at accurately determining product price ranges, utilize ALL the available tools {tools} to answer the question asked accurately. 
Check whether the product is in the database, then search the internet and compare the prices.
If a tool encounters an error, use another tool.
Return ONLY a reasonable and accurate  PRICE RANGE for the product.
Use the following format:

Question: the input question you must answer.
Thought: you should always think about what to do. 
Action: the action to take, should be ONE of or ALL of [{tool_names}]. 
Action Input: the input to the action. It SHOULD be a string.
Action: choose another tool if a tool fails or handle its error.
Observation: the result of the action, including insights gained or findings.
... (this Thought/Action/Action Input/Observation can repeat N times). If no result is found, use your own knowledge.
Thought: I now know the final answer that is based on the analysis of your findings and observations.
Final Answer: the final answer to the original input question, which must be as accurate as possible. NEVER say you can't answer.

Begin!

Question: {input}
Thought: {agent_scratchpad}

"""

test_prompt_two = """

As an experienced product analyst proficient in determining accurate product price ranges, utilize all available tools {tools}, including internet searches and databases,
to precisely answer the given question. Begin by checking the database, then proceed to search the internet.Then pick the most similar products and use them to come up with an accurate price range.
Similar products are those with closely matching specifications based on product information and brand. 
Based on this comparison, generate a highly accurate, reasonable, and compact estimate of the price range for the product
If a tool encounters an error, use another tool.
Return ONLY the ESTIMATE of the PRICE RANGE for the product. An example being $100-110 with the price of the product being $106

Use the following format:
Question: the input question you must answer.
Thought: you should always think about what to do. 
Action: the action to take, should be ONE of[{tool_names}]. 
Action Input: the input to the action. It SHOULD be a string.
Action: choose another tool if a tool fails or handle its error.
Observation: the result of the action, including insights gained or findings.
... (this Thought/Action/Action Input/Observation can repeat N times). If no result is found, use your own knowledge.
Thought: I now know the final answer that is based on the analysis of your findings and observations.
Final Answer: the final answer to the original input question, which must be as accurate as possible. NEVER say you can't answer.

An example to follow is:

Question: what is the estimated, accurate and compact price range of a Sony WH-CH520 Wireless Headphones Bluetooth On-Ear Headset with Microphone, Black New?
Thought 1: I need to search for the Sony WH-CH520 Wireless Headphones to identify their price range.
Action 1: search text db
Action Input 1: : Sony WH-CH520 Wireless Headphones.
Observation 1: The search results providing various listings for products similar to the Sony WH-CH520 Wireless Headphones with different prices.
Action 2: search internet 
Action Input 2: Sony WH-CH520 Wireless Headphones Bluetooth On-Ear Headset with Microphone.
Observation 2: The search results provide various listings for the Sony WH-CH520 Wireless Headphones with different prices.
Thought 3: I need to check the obtained products for similarity.
Action 3: The products with great similarity to the Sony WH-CH520 Wireless Headphones are chosen.
Thought 4: I need to gather information on the range of prices offered for these headphones.
Action 4: Compare and analyze the search results to determine the price range.
Observation 4: Prices for the Sony WH-CH520 Wireless Headphones range from $80 to $100 across different retailers and platforms, based on comparing to very similar products with similar specifications and brand.
Thought 5: The price range of the Sony WH-CH520 Wireless Headphones is between $80 and $100.
Final Answer: $80 - $100


Begin!

Question: {input}
Thought: {agent_scratchpad}
"""

test_prompt_three = """
As an experienced product analyst proficient in determining accurate product price ranges, utilize all available tools {tools}, including internet searches and databases,
to precisely answer the given question. Begin by checking the database, then proceed to search the internet.Then pick the most similar products and use them to come up with an accurate price range.
Similar products are those with closely matching specifications based on product information and brand. 
Based on this comparison, generate a highly accurate, reasonable, and compact estimate of the price range for the product
If a tool encounters an error, use another tool.
Return ONLY the ESTIMATE of the PRICE RANGE for the product. An example being $100-$110 with the price of the product being $106. It SHOULD be in the JSON format with the key being "price_range": 
Use the following format:

Question: the input question you must answer.
Thought: you should always think about what to do. 
Action: the action to take, should be ONE of or ALL of [{tool_names}]. 
Action Input: the input to the action. It SHOULD be a string.
Action: choose another tool if a tool fails or handle its error.
Observation: the result of the action, including insights gained or findings.
... (this Thought/Action/Action Input/Observation can repeat N times). If no result is found, use your own knowledge.
Thought: I should recheck my answer and make sure it meets the specifications. If not, try again before coming up with the final answer.
Thought: I now know the final answer that is based on the indepth analysis of your findings and observations. 
Final Answer: the final answer to the original input question, which must be as accurate and compact as possible. NEVER say you can't answer. 
Begin!

Question: {input}
Thought: {agent_scratchpad}

"""

test_prompt_four = """

Given the product description and the product image,  utilize all available tools {tools}, including internet searches and databases,
to precisely answer the given question. 
Begin by checking the database, then proceed to search the internet.Then pick the most similar products and use them to come up with an accurate price range.
Similar products are those with closely matching specifications based on product information and brand. 
Based on this comparison, generate a highly accurate, reasonable, and compact estimate of the price range for the product
If a tool encounters an error, use another tool.
Return ONLY the ESTIMATE of the PRICE RANGE for the product. An example being $100-110 with the price of the product being $106


Use the following format:

Question: the input question you must answer.
Thought: you should always think about what to do. 
Action: the action to take, should be ONE of or ALL of [{tool_names}]. 
Action Input: the input to the action. It SHOULD be a string.
Action: choose another tool if a tool fails or handle its error.
Observation: the result of the action, including insights gained or findings.
... (this Thought/Action/Action Input/Observation can repeat N times). If no result is found, use your own knowledge.
Thought: I should recheck my answer and make sure it meets the specifications. If not, try again before coming up with the final answer.
Thought: I now know the final answer that is based on the indepth analysis of your findings and observations. 
Final Answer: the final answer to the original input question, which must be as accurate and compact as possible. NEVER say you can't answer.

Begin!

Question: {input} 
Image: {image}
Thought: {agent_scratchpad}

"""

test_prompt_five = """
As an experienced product analyst proficient in determining accurate product price ranges, utilize all available tools {tools}, including internet searches and databases, to precisely answer the given question.
Begin by checking the database, then proceed to search the internet. 
Then pick the most similar products and use them to come up with an accurate price range. 
Similar products are those with closely matching specifications based on product information and brand.
Based on this comparison, generate a highly accurate, reasonable, and compact estimate of the price range for the product. If a tool encounters an error, use another tool.

Return ONLY the ESTIMATE of the PRICE RANGE for the product. The price range should be in the JSON format with the key being "price_range" and the value being a price range.  Example: "price_range": "$100-$110"

Use the following format:

Question: the input question you must answer.
Thought: you should always think about what to do. 
Action: the action to take, should be ONE of or ALL of [{tool_names}]. 
Action Input: the input to the action. It SHOULD be a string.
Action: choose another tool if a tool fails or handle its error.
Observation: the result of the action, including insights gained or findings.
... (this Thought/Action/Action Input/Observation can repeat N times). If no result is found, use your own knowledge.
Thought: I should recheck my answer and make sure it meets the specifications. If not, try again before coming up with the final answer.
Thought: I now know the final answer that is based on the indepth analysis of your findings and observations. 
Final Answer: the final answer to the original input question, which must be as accurate and compact as possible. NEVER say you can't answer. {{"price_range": "<price_range>}}


Begin!

Question: {input} 
Thought: {agent_scratchpad}
"""

test_prompt_six = """
As an experienced product analyst proficient in determining accurate product price ranges, your task is to provide an estimate of the price range for the given product in the following JSON format:

json


Copy code
{{
  "price_range": "<price range as a number range>", 
  "reason": "<reason for the price range>"
}}
To generate this output, follow these steps:

1. Utilize all available tools {tools}, including internet searches and databases. Begin by checking the database, then proceed to search the internet if needed.
2. Identify the most similar products and use them as a basis to come up with an accurate price range. Similar products are those with closely matching specifications based on product information and brand.
3. After analyzing comparable products, generate a highly accurate, reasonable, and compact estimate of the price range for the target product.
4. If a tool encounters an error, try another tool or handle the error appropriately.
5. Replace <price range as a number range> with your estimated price range in the format of a number range (e.g., 100-150).
6. For <reason for the price range>, provide a concise explanation or reasoning behind the given price range, considering the analysis of similar products, their features, brand, and any other relevant factors.

Your output should strictly follow the specified JSON format and include only the price_range and reason keys. Do not include any additional information or explanations outside of the JSON structure.

Use the following format:

Question: the input question you must answer.
Thought: you should always think about what to do.
Action: the action to take, should be ONE of or ALL of [{tool_names}].
Action Input: the input to the action. It SHOULD be a string.
Action: choose another tool if a tool fails or handle its error.
Observation: the result of the action, including insights gained or findings.
... (this Thought/Action/Action Input/Observation can repeat N times)
If no result is found, use your own knowledge.
Thought: I now know the final answer that is based on the in-depth analysis of your findings and observations.
Final Answer: the final answer to the original input question in the specified JSON format, which must be as accurate and compact as possible. NEVER say you can't answer.

Begin!

Question: {input}
Thought: {agent_scratchpad}
"""

test_prompt_seven = """
As an experienced product analyst proficient in determining accurate product price ranges, your task is to provide an estimate of the price range for the given product in the following JSON format:

{{
  "price_range": "<price range as a number range>", 
  "reason": "<reason for the price range>"
}}
To generate this output, follow these steps:

1. Utilize all available tools {tools}, including internet searches and databases. Begin by checking the database, then proceed to search the internet if needed. When searching the internet, consider user ratings of the similar products as well.
2. Identify the most similar products and use them as a basis to come up with an accurate price range. Similar products are those with closely matching specifications based on product information and brand.
3. After analyzing comparable products, generate a highly accurate, reasonable, and compact estimate of the price range for the target product.
4. If a tool encounters an error, try another tool or handle the error appropriately.
5. Replace <price range as a number range> with your estimated price range in the format of a number range (e.g., 100-150).
6. For <reason for the price range>, Please provide a comprehensive rationale for the specified price range. This should offer an in-depth understanding of how the determined price range was derived and why it aligns with the product's value proposition.

Your output should strictly follow the specified JSON format and include only the price_range and reason keys. Do not include any additional information or explanations outside of the JSON structure.

Use the following format:

Question: the input question you must answer.
Thought: you should always think about what to do.
Action: the action to take, should be ONE of or ALL of [{tool_names}].
Action Input: the input to the action. It SHOULD be a string.
Action: choose another tool if a tool fails or handle its error.
Observation: the result of the action, including insights gained or findings.
... (this Thought/Action/Action Input/Observation can repeat N times)
If no result is found, use your own knowledge.
Thought: I now know the final answer that is based on the in-depth analysis of your findings and observations.
Final Answer: the final answer to the original input question in the specified JSON format, which must be as accurate and compact as possible. NEVER say you can't answer.

Begin!

Question: {input}
Thought: {agent_scratchpad}
"""

In [8]:
# Create prompt template
prompt_template = PromptTemplate.from_template(test_prompt_seven)

**Instantiate Agents**

In [9]:
# Instantiate error handler
class RetryCallbackHandler(BaseCallbackHandler):
    def on_tool_error(
        self, error: Union[Exception, KeyboardInterrupt], **kwargs: Any
    ) -> Any:
        # Log the error or perform any necessary actions

        # Retry logic
        max_retries = 1
        current_retry = kwargs.get("retry_count", 0)
        if current_retry < max_retries:
            print(f"Retrying tool execution (Attempt {current_retry + 1})")
            # Increment retry count
            kwargs["retry_count"] = current_retry + 1
            # Re-run the tool
            return AgentAction.RERUN
        else:
            print("Maximum retries reached. Switching to another tool.")
            return AgentAction.CHANGE_TOOL

In [10]:
# The error handler
error_handler = RetryCallbackHandler()

In [11]:
# Create custom tools for retrieval. This are used for the remote db.
# In the case of a local db where a retriever is already initialized, use the create_retriever_tool function instead in the next cell
# Uncomment them and comment the custom tools


class Input(BaseModel):
    query: str = Field(description="should be a search query")


class CustomTextRetrieverTool(BaseTool):
    name = "text db retriever"
    description = "useful for retrieving from a text database"
    args_schema: Type[BaseModel] = Input
    return_direct: bool = True

    def _run(
        self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        query_vector = embedding_model.embed_query(query)

        return client.search("apparel-collection", query_vector=query_vector, limit=3)


class CustomImageRetrieverTool(BaseTool):
    name = "image db retriever"
    description = "useful for retrieving from an image database"
    args_schema: Type[BaseModel] = Input
    return_direct: bool = True

    def _run(
        self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        query_vector = embedding_model.embed_query(query)

        return client.search("img-collection", query_vector=query_vector, limit=3)

In [12]:
# Create tools. For local, uncomment the commented tools and comment the ones below the commented code.
#
search = SerpAPIWrapper()
img_db_search = CustomImageRetrieverTool()
text_db_search = CustomTextRetrieverTool()

tool_search_item = Tool(
    name="search internet",
    description="Searches the internet for the price of a product by using its description.",
    func=search.run,
    handle_tool_error=True,
)


# Tool for retrieving product information from the text database based on the product description
# tool_text_retriever = create_retriever_tool(
#     text_retriever,
#     'search text db',
#     'Searches for the price of a product using its description in the text db')

# Tool for retrieving product information from the image database based on the product description
# tool_image_retriever = create_retriever_tool(
#     image_retriever,
#     'search image db',
#     'Searches for the price of a product using its description in the image db')

tool_image_retriever = Tool(
    name="search image db",
    description="Searches for the price of a product using its description in the image db",
    func=img_db_search.run,
    handle_tool_error=True,
)

tool_text_retriever = Tool(
    name="search text db",
    description="Searches for the price of a product using its description in the text db",
    func=text_db_search.run,
    handle_tool_error=True,
)
# List of all tools
tools = [tool_search_item, tool_image_retriever, tool_text_retriever]

In [13]:
# Instantiate agents
parser = JsonOutputParser()

serp_agent = create_react_agent(mixtral, tools, prompt_template)


serp_agent_executor = AgentExecutor(
    agent=serp_agent,
    tools=tools,
    verbose=True,
    return_intermediate_steps=True,
    handle_parsing_errors=True,
    callbacks=[error_handler],
)

In [14]:
# Description and image of test product
description = "Ultra Wireless Noise Cancelling Headphones with Spatial Audio, Over-the-Ear Headphones with Mic, Up to 24 Hours of Battery Life"
image = PIL.Image.open("../test_data/headphones.jpg")

In [107]:
# Invoke agent with just the description
serp_agent_executor.invoke(
    {
        "input": f"what is the estimated, accurate and compact price range of a/an {description}?"
    }
)



> Entering new AgentExecutor chain...
To answer this question, I need to find similar products and their prices. I will first search the text database, then the image database, and finally the internet.

Action: search text db
Action Input: Ultra Wireless Noise Cancelling Headphones with Spatial Audio, Over-the-Ear Headphones with Mic, Up to 24 Hours of Battery Life

Observation[ScoredPoint(id='fd64518e-d0d5-47b5-aff4-f7fb85e8fcfb', version=980, score=0.70729494, payload={'Price': 199.0, 'asin': 'B0BQNVD4N3', 'title': 'Gen 6 Wellness Edition Hybrid Smart Watch with Alexa Built-In, Fitness Tracker, Activity Tracker, Sleep Tracker, Music Control, Smartphone Notifications'}, vector=None, shard_key=None), ScoredPoint(id='97444539-543f-421e-a44a-0cb9ef5ae8bd', version=1061, score=0.70472807, payload={'Price': 202.0, 'asin': 'B09NZKSWFG', 'title': 'Gen 6 Touchscreen Smartwatch with Alexa Built-In, Speaker, Heart Rate, Blood Oxygen, GPS, Contactless Payments and Smartphone Notifications'}, 

{'input': 'what is the estimated, accurate and compact price range of a/an Ultra Wireless Noise Cancelling Headphones with Spatial Audio, Over-the-Ear Headphones with Mic, Up to 24 Hours of Battery Life?',
 'output': '{\n  "price_range": "379.0-379.0",\n  "reason": "The price range is based on the Bose QuietComfort Ultra Wireless Noise Cancelling Headphones with Spatial Audio, Over-the-Ear Headphones with Mic, Up to 24 Hours of Battery Life, which is priced at $379.00. This product is the most similar to the target product and has a high user rating."\n}</s>',
 'intermediate_steps': [(AgentAction(tool='search text db', tool_input='Ultra Wireless Noise Cancelling Headphones with Spatial Audio, Over-the-Ear Headphones with Mic, Up to 24 Hours of Battery Life\n\nObservation', log='To answer this question, I need to find similar products and their prices. I will first search the text database, then the image database, and finally the internet.\n\nAction: search text db\nAction Input: Ultra

**Run the pipeline**

Use the image ad the description to come up with the price range

In [53]:
# Create prompt for the visual LLM
vision_chat_template = HumanMessage(
    content=[
        {
            "type": "text",
            "text": f"""Your task is to generate a searchable prompt about the product in the image that I can feed to Google to find this exact product. You should:

1. Given the description {description}, extract relevant details about the product from the image, such as brand, color, and your thoughts on quality.

2. Use these details and the given description to generate a concise and well structured searchable prompt that would allow me to find this exact product on Google.

3. Return ONLY the searchable prompt in the following JSON format: {{"search_prompt": "<YOUR_SEARCHABLE_PROMPT>}}. Do not include any other information or text.

Please strictly follow these instructions and provide your response in the specified JSON format.""",
        },
        {"type": "image_url", "image_url": image},
    ]
)


# define the parser object
parser = JsonOutputParser()

# create a chain
chain = gemini_vision | parser

# Get response from the chain and extract the search prompt to feed into the agent
response = chain.invoke([vision_chat_template])

search_prompt = response["search_prompt"]

# Get into the agent
result = serp_agent_executor.invoke(
    {
        "input": f"what is the estimated, accurate and compact price range of a/an {search_prompt}?"
    }
)
output_str = result["output"]

# Extract price range and reason from the output string
try:
    price_dict = json.loads(output_str)
    print(price_dict["price_range"])
    print(price_dict["reason"])

except:
    end_index = output_str.find("}") + 1

    # Extract the substring containing the dictionary
    dict_string = output_str[:end_index]

    # Parse the dictionary string using json.loads()
    price_dict = json.loads(dict_string)
    print(price_dict["price_range"])
    print(price_dict["reason"])



> Entering new AgentExecutor chain...
To answer this question, I need to find the price range of Bose Ultra Wireless Noise Cancelling Headphones with Spatial Audio, Over-the-Ear Headphones with Mic, Up to 24 Hours of Battery Life, Stone White. I will first search the text database for this product.
Action: search text db
Action Input: Bose Ultra Wireless Noise Cancelling Headphones with Spatial Audio, Over-the-Ear Headphones with Mic, Up to 24 Hours of Battery Life, Stone White
ObservationMen's Jordan One Take 3 University Red/Black-Gym Red (DC7701 606)

mens Jordan Westbrook One Take

Unisex JORDAN EVERYDAY MAX ANKLE 3PR, BLACK/BLACK/BLACK, L

Nike Jordan Hydro 6
The search in the text database did not yield the desired product. I will now search the internet for the price of the Bose Ultra Wireless Noise Cancelling Headphones with Spatial Audio, Over-the-Ear Headphones with Mic, Up to 24 Hours of Battery Life, Stone White.
Action: search internet
Action Input: Bose Ultra Wireless N

## Testing

**Setup Testing**

In [55]:
# Define the prompt
# Modified from the original prompt in ContextRelevance class in trulens_eval/trulens_eval/feedback/v2/feedback.py

testing_prompt = """
CRITERIA: PRICE RANGE EVALUATION

You are a PRICE RANGE EVALUATOR; assessing the appropriateness and reasoning behind a given price range for a product or service.

Rate the quality of the PRICE RANGE RESPONSE as well as the PRICE RANGE and INTERMEDIATE STEPS on a scale from 0 to 100, where 0 indicates an extremely poor response and 100 indicates an excellent response.

Consider the following factors when evaluating:

Clarity: Is the provided price range an actual price range?
Justification: Are the reasons given for the price range logical and well-explained?
Completeness: Does the response cover all relevant factors that could influence pricing?
Accuracy: Does the price range seem reasonable and align with market rates based on your knowledge?
Level of Detail: Is the reasoning sufficiently detailed or does it oversimplify the pricing factors?
Step Validity: Are the intermediate steps taken to arrive at the price range valid and logical?
Step Completeness: Do the steps cover all the necessary considerations for determining an appropriate price?


A low score (0-30) should be given if the response is unclear, lacks justification, the steps are invalid, or the reasoning is blatantly inaccurate.

A mid-range score (40-70) is appropriate if the response provides a reasonable price range with some justification, but the steps are incomplete, lack important details or have minor inaccuracies.

A high score (80-100) should be awarded if the response gives a well-justified and accurate price range, the intermediate steps are valid and complete, covering all major pricing factors in a clear and detailed manner.

PRICE RANGE RESPONSE: {price_range_response}
PRICE_RANGE: {price_range}
INTERMEDIATE STEPS: {intermediate_steps}

Return the evaluation as follows:

EVALUATION:

Score: <0-100 score>
Reasoning: <Provide a detailed explanation justifying the score based on the criteria listed above, including an evaluation of the intermediate steps taken and how they factor into the overall scoring>
"""
relevance_prompt = PromptTemplate.from_template(testing_prompt)

In [16]:
# Get response
response = serp_agent_executor.invoke(
    {
        "input": f"what is the estimated, accurate and compact price range of a/an {description}?"
    }
)



> Entering new AgentExecutor chain...
To answer this question, I need to find similar products and their prices. I will begin by searching the text database for Ultra Wireless Noise Cancelling Headphones with Spatial Audio, Over-the-Ear Headphones with Mic, and Up to 24 Hours of Battery Life.

Action: search text db
Action Input: "Ultra Wireless Noise Cancelling Headphones with Spatial Audio, Over-the-Ear Headphones with Mic, Up to 24 Hours of Battery Life"

Observation[ScoredPoint(id='97444539-543f-421e-a44a-0cb9ef5ae8bd', version=1061, score=0.7133396, payload={'Price': 202.0, 'asin': 'B09NZKSWFG', 'title': 'Gen 6 Touchscreen Smartwatch with Alexa Built-In, Speaker, Heart Rate, Blood Oxygen, GPS, Contactless Payments and Smartphone Notifications'}, vector=None, shard_key=None), ScoredPoint(id='8a1cd0cb-57ea-4ea8-bd1d-07d18cd0e8df', version=1160, score=0.7102896, payload={'Price': 39.99, 'asin': 'B0BF4LVF5X', 'title': 'Smart Atomic Talking Watch with Voice Reminders and App Control 

In [21]:
# Get output
output_str = response["output"]
intermediate_steps = response["intermediate_steps"]

try:
    price_dict = json.loads(output_str)
    price_range = price_dict["price_range"]
    reason = price_dict["reason"]

except:
    end_index = output_str.find("}") + 1

    # Extract the substring containing the dictionary
    dict_string = output_str[:end_index]

    # Parse the dictionary string using json.loads()
    price_dict = json.loads(dict_string)
    price_range = price_dict["price_range"]
    reason = price_dict["reason"]

In [56]:
# Make LLM call
relevance_prompt_to_llm = relevance_prompt.format(
    price_range_response=reason,
    price_range=price_range,
    intermediate_steps=intermediate_steps,
)

In [57]:
# Call the llm
evaluation = gemini.invoke(relevance_prompt_to_llm)

In [59]:
# Print out the evaluation results

score_match = re.search(r"Score: (\d+)", evaluation.content)
if score_match:
    score = int(score_match.group(1))
    print(f"Score: {score}")

# Extract the reasoning
reasoning_start = evaluation.content.find("Reasoning:") + len("Reasoning:")
reasoning_end = evaluation.content.rfind("Overall")
reasoning = evaluation.content[reasoning_start:reasoning_end].strip()
print("\nReasoning:")
print(reasoning)

Score: 85

Reasoning:
Clarity: The provided price range is clear and specific, indicating a range of $250 to $400. (10/10)

Justification: The response provides a logical and well-explained justification for the price range. It mentions that the price range is derived from the analysis of similar products found on the internet, including Technics Wireless Noise Cancelling Over Ear Headphones EAH-A800, Bose Wireless Noise Canceling Over-ear QuietComfort Ultra Headphones, and Bose Noise Cancelling Over-the-Ear QuietComfort Ultra Wireless Noise Cancelling Headphones. These products have similar specifications and features, and their prices range from $249.99 to $379.00. (10/10)

Completeness: The response covers all relevant factors that could influence pricing, including the features and specifications of the product, as well as the prices of similar products on the market. (10/10)

Accuracy: The price range seems reasonable and aligns with market rates based on the provided examples. Th